In [1]:
from DataLoader import DataLoader
import GraphManager
import GNN 
import Learning
import numpy as np
import pandas as pd
import torch
import torch.optim
import torch_geometric
import torch_geometric.data
import networkx as nx
import matplotlib.pyplot as plt
from Config import GLOBALS, HYPERPARAETERS
from tqdm import tqdm
import pickle
import json
import torchinfo

In [2]:
dl = DataLoader('data/KaggleDataset_withBO.csv', 11) 
gm = GraphManager.GraphManager(dl, GLOBALS.DEVICE.value)

In [3]:
if not GLOBALS.already_saved.value:
    for league, league_df in dl.dataset.groupby('league', sort=False):
        print(f'Making {league} Graphs...')
        gm.make(
            league_df,
            mode='CG',
            validation_portion=HYPERPARAETERS.ValidationPortion.value,
            test_portion=HYPERPARAETERS.TestPortion.value,
            window_size=50,
            saveto=f'{GLOBALS.SavePath.value}{league}.gm'
        )
        print(f'Graph List Length: {len(gm.graph_list)}')
        print(f'Train Mask: {len(gm.train_mask)}')
        print(f'Validation Mask: {len(gm.validation_mask)}')
        print(f'Test Mask: {len(gm.test_mask)}')
        print(f'Saved in: {GLOBALS.SavePath.value}{league}.gm')
        print('='*80)

In [4]:
if GLOBALS.continue_training.value:
    model = torch.load(f'{GLOBALS.LoadPath.value}model.pth')
    with open(f'{GLOBALS.LoadPath.value}lists.pl', 'rb') as pf:
        loss_list, train_acc_list, eval_acc_list = pickle.load(pf)
else:
    model = GNN.HeteroGNN(
        embedding_dims=[dl.entities.shape[0], HYPERPARAETERS.EmbeddingDim.value],
        conv_dims=HYPERPARAETERS.ConvDims.value,
        fc_dims=HYPERPARAETERS.FCDims.value,
        dropout=HYPERPARAETERS.DropOuts.value,
        classify=True
    ).to(GLOBALS.DEVICE.value)
    model.reset_parameters()

    with open(f'{GLOBALS.SavePath.value}HyperParameters.json', 'w') as fp:
        json.dump({name: member.value for name, member in HYPERPARAETERS.__members__.items()}, fp, indent=4)
    loss_list = []
    train_acc_list = []
    eval_acc_list = []


criterion = torch.nn.NLLLoss()
# criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value) 

In [5]:
torchinfo.summary(model, depth=4)

Layer (type:depth-idx)                   Param #
HeteroGNN                                --
├─Embedding: 1-1                         19,532
├─ModuleList: 1-2                        --
│    └─HeteroConv: 2-1                   --
│    │    └─ModuleDict: 3-1              --
│    │    │    └─GCNConv: 4-1            12
│    │    │    └─GCNConv: 4-2            12
│    │    │    └─GCNConv: 4-3            12
│    │    │    └─GATConv: 4-4            28
│    │    │    └─GATConv: 4-5            28
│    │    │    └─GCNConv: 4-6            12
│    │    │    └─GCNConv: 4-7            12
│    │    │    └─GCNConv: 4-8            12
│    │    │    └─GCNConv: 4-9            12
│    └─HeteroConv: 2-2                   --
│    │    └─ModuleDict: 3-2              --
│    │    │    └─GCNConv: 4-10           60
│    │    │    └─GCNConv: 4-11           60
│    │    │    └─GCNConv: 4-12           60
│    │    │    └─GATConv: 4-13           132
│    │    │    └─GATConv: 4-14           132
│    │    │    └─GCNC

In [6]:
print('*********************************** Phase 1 ***********************************')
prev_best_val_path = None
for round in range(HYPERPARAETERS.Phase1Rounds.value):
    print(f'############################## Round {round + 1} ##############################')
    for league, league_df in dl.dataset.groupby('league', sort=False):
        print(f'Training On: {league}')
        if prev_best_val_path is not None:
            model = torch.load(prev_best_val_path)
            optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
        if GLOBALS.already_saved.value:
            gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
        else:
            gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
        try:
            max_acc = 0
            for epoch in range(HYPERPARAETERS.Phase1Epochs.value):
                loss, train_acc = Learning.train(
                    model=model,
                    graph_list=gm.graph_list,
                    train_indcs=gm.train_mask,
                    mode='RP',
                    criterion=criterion,
                    optimizer=optimizer,
                )

                val_acc = Learning.evaluation(
                    model=model,
                    graph_list=gm.graph_list,
                    eval_indcs=gm.validation_mask,
                    mode='RP'
                )

                print(f'=================================== EPOCH {epoch + 1} ===================================')
                print(f'Average Loss: {loss} - Train Accuracy: {train_acc: .3f} - Validation Accuracy: {val_acc: .3f}')

                loss_list.append(loss)
                train_acc_list.append(train_acc)
                eval_acc_list.append(val_acc)

                if val_acc >= max_acc:
                    max_acc = val_acc
                    torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_MaxVal_Ph1.pth')
                    prev_best_val_path = f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_MaxVal_Ph1.pth'

                if (epoch+1) % GLOBALS.SaveEvery.value == 0:
                    torch.save(model, f'{GLOBALS.SavePath.value}model.pth')
                    with open(f'{GLOBALS.SavePath.value}lists.pl', 'wb') as pf:
                        pickle.dump((loss_list, train_acc_list, eval_acc_list), pf)
        except KeyboardInterrupt:
            pass
        torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_Ph1.pth')
    torch.save(model, f'{GLOBALS.SavePath.value}model_All_R{round+1}_Ph1.pth')

*********************************** Phase 1 ***********************************
############################## Round 1 ##############################
Training On: England Premier League
=================================== EPOCH 1 ===================================
Average Loss: 1.206635359628701 - Train Accuracy:  0.376 - Validation Accuracy:  0.429
=================================== EPOCH 2 ===================================
Average Loss: 1.0934516024687653 - Train Accuracy:  0.417 - Validation Accuracy:  0.442
=================================== EPOCH 3 ===================================
Average Loss: 1.076762010776457 - Train Accuracy:  0.455 - Validation Accuracy:  0.442
=================================== EPOCH 4 ===================================
Average Loss: 1.0694859859384136 - Train Accuracy:  0.457 - Validation Accuracy:  0.442
=================================== EPOCH 5 ===================================
Average Loss: 1.0671977008321158 - Train Accuracy:  0.460 - Vali

In [7]:
acc_dict = {(f'R{round+1}', 'League'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)}
acc_dict.update({(f'R{round+1}', 'MaxVal'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)})
acc_dict.update({(f'R{round+1}', 'All'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)})
league_names = dl.dataset['league'].unique().tolist()
league_count = len(league_names)
for i, (league, league_df) in enumerate(dl.dataset.groupby('league', sort=False)):
    if GLOBALS.already_saved.value:
        gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
    for round in range(HYPERPARAETERS.Phase1Rounds.value):
        tm = torch.load(f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_Ph1.pth')
        acc_on_league = Learning.evaluation(tm, gm.graph_list, gm.validation_mask, 'RP')
        acc_dict[(f'R{round+1}', 'League')].append(acc_on_league)

        tm = torch.load(f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_MaxVal_Ph1.pth')
        acc_on_maxval = Learning.evaluation(tm, gm.graph_list, gm.validation_mask, 'RP')
        acc_dict[(f'R{round+1}', 'MaxVal')].append(acc_on_maxval)
        
        tm = torch.load(f'{GLOBALS.SavePath.value}model_All_R{round+1}_Ph1.pth')
        acc_on_all = Learning.evaluation(tm, gm.graph_list, gm.validation_mask, 'RP')
        acc_dict[(f'R{round+1}', 'All')].append(acc_on_all)
        
eval_acc_df = pd.DataFrame(acc_dict, index=league_names).sort_index(axis='columns', level=0)
eval_acc_df

R1                            R2            \
                               All    League    MaxVal       All    League   
England Premier League    0.462585  0.523810  0.527211  0.455782  0.476190   
France Ligue 1            0.485915  0.496479  0.510563  0.471831  0.503521   
Italy Serie A             0.468401  0.475836  0.490706  0.472119  0.494424   
Spain LIGA BBVA           0.500000  0.525362  0.536232  0.503623  0.532609   
Germany 1. Bundesliga     0.475000  0.500000  0.516667  0.475000  0.487500   
Netherlands Eredivisie    0.497608  0.526316  0.535885  0.511962  0.526316   
Scotland Premier League   0.554140  0.547771  0.573248  0.585987  0.560510   
Portugal Liga ZON Sagres  0.490196  0.503268  0.522876  0.503268  0.509804   
Belgium Jupiler League    0.510791  0.510791  0.517986  0.517986  0.517986   

                                    
                            MaxVal  
England Premier League    0.510204  
France Ligue 1            0.521127  
Italy Serie A             0.501859  
Spain LIGA BBVA           0.554348  
Germany 1. Bundesliga     0.504167  
Netherlands Eredivisie    0.540670  
Scotland Premier League   0.579618  
Portugal Liga ZON Sagres  0.542484  
Belgium Jupiler League    0.539568

In [8]:
acc_dict = {(f'R{round+1}', 'League'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)}
acc_dict.update({(f'R{round+1}', 'MaxVal'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)})
acc_dict.update({(f'R{round+1}', 'All'): [] for round in range(HYPERPARAETERS.Phase1Rounds.value)})
league_names = dl.dataset['league'].unique().tolist()
league_count = len(league_names)
for i, (league, league_df) in enumerate(dl.dataset.groupby('league', sort=False)):
    if GLOBALS.already_saved.value:
        gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
    for round in range(HYPERPARAETERS.Phase1Rounds.value):
        tm = torch.load(f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_Ph1.pth')
        acc_on_league = Learning.evaluation(tm, gm.graph_list, gm.test_mask, 'RP')
        acc_dict[(f'R{round+1}', 'League')].append(acc_on_league)

        tm = torch.load(f'{GLOBALS.SavePath.value}model_{league}_R{round+1}_MaxVal_Ph1.pth')
        acc_on_maxval = Learning.evaluation(tm, gm.graph_list, gm.test_mask, 'RP')
        acc_dict[(f'R{round+1}', 'MaxVal')].append(acc_on_maxval)
        
        tm = torch.load(f'{GLOBALS.SavePath.value}model_All_R{round+1}_Ph1.pth')
        acc_on_all = Learning.evaluation(tm, gm.graph_list, gm.test_mask, 'RP')
        acc_dict[(f'R{round+1}', 'All')].append(acc_on_all)
        

test_acc_df = pd.DataFrame(acc_dict, index=league_names).sort_index(axis='columns', level=0)
test_acc_df

R1                            R2            \
                               All    League    MaxVal       All    League   
England Premier League    0.395425  0.424837  0.434641  0.401961  0.415033   
France Ligue 1            0.469751  0.483986  0.491103  0.455516  0.469751   
Italy Serie A             0.507299  0.500000  0.507299  0.496350  0.518248   
Spain LIGA BBVA           0.555556  0.545139  0.555556  0.545139  0.552083   
Germany 1. Bundesliga     0.460317  0.488095  0.492063  0.468254  0.484127   
Netherlands Eredivisie    0.444976  0.464115  0.459330  0.454545  0.473684   
Scotland Premier League   0.449367  0.436709  0.455696  0.462025  0.443038   
Portugal Liga ZON Sagres  0.500000  0.517647  0.535294  0.494118  0.523529   
Belgium Jupiler League    0.453237  0.453237  0.460432  0.460432  0.460432   

                                    
                            MaxVal  
England Premier League    0.405229  
France Ligue 1            0.491103  
Italy Serie A             0.510949  
Spain LIGA BBVA           0.552083  
Germany 1. Bundesliga     0.472222  
Netherlands Eredivisie    0.454545  
Scotland Premier League   0.430380  
Portugal Liga ZON Sagres  0.523529  
Belgium Jupiler League    0.474820

In [11]:
eval_acc_df.apply(lambda z: z.idxmax(), axis=1)

England Premier League      (R1, MaxVal)
France Ligue 1              (R2, MaxVal)
Italy Serie A               (R2, MaxVal)
Spain LIGA BBVA             (R2, MaxVal)
Germany 1. Bundesliga       (R1, MaxVal)
Netherlands Eredivisie      (R2, MaxVal)
Scotland Premier League        (R2, All)
Portugal Liga ZON Sagres    (R2, MaxVal)
Belgium Jupiler League      (R2, MaxVal)
dtype: object

In [13]:
test_acc_df.apply(lambda z: z.idxmax(), axis=1)

England Premier League      (R1, MaxVal)
France Ligue 1              (R1, MaxVal)
Italy Serie A               (R2, League)
Spain LIGA BBVA                (R1, All)
Germany 1. Bundesliga       (R1, MaxVal)
Netherlands Eredivisie      (R2, League)
Scotland Premier League        (R2, All)
Portugal Liga ZON Sagres    (R1, MaxVal)
Belgium Jupiler League      (R2, MaxVal)
dtype: object

In [14]:
test_acc_df.apply(lambda z: z.max(), axis=1)

England Premier League      0.434641
France Ligue 1              0.491103
Italy Serie A               0.518248
Spain LIGA BBVA             0.555556
Germany 1. Bundesliga       0.492063
Netherlands Eredivisie      0.473684
Scotland Premier League     0.462025
Portugal Liga ZON Sagres    0.535294
Belgium Jupiler League      0.474820
dtype: float64

In [15]:
test_acc_df.apply(lambda z: z.max(), axis=1) - [row[eval_acc_df.apply(lambda z: z.idxmax(), axis=1)[idx]] for idx, row in test_acc_df.iterrows()]

England Premier League      0.000000
France Ligue 1              0.000000
Italy Serie A               0.007299
Spain LIGA BBVA             0.003472
Germany 1. Bundesliga       0.000000
Netherlands Eredivisie      0.019139
Scotland Premier League     0.000000
Portugal Liga ZON Sagres    0.011765
Belgium Jupiler League      0.000000
dtype: float64

In [46]:
val_dic = {
    'epochs': [0] * eval_acc_df.shape[0],
    'accuracy': [0] * eval_acc_df.shape[0],
}
val_dic.update({e: [0] * eval_acc_df.shape[0] for e in [0, 1, 2, 3]})
chosen_epochs = pd.DataFrame(
    val_dic, 
    index=eval_acc_df.index
)

for league, (best_R, best_type) in eval_acc_df.apply(lambda z: z.idxmax(), axis=1).iteritems():
    print(f'Incremental Validation on league: {league}')
    if GLOBALS.already_saved.value:
        gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
    for epochs in [0, 1, 2, 3]:
        model = torch.load(f'{GLOBALS.SavePath.value}model_{league if best_type in ["League", "MaxVal"] else "All"}_{best_R}{"_MaxVal" if best_type == "MaxVal" else ""}_Ph1.pth')
        optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
        try:
            train_list = gm.train_mask.copy()
            t_correct = 0
            t_total = 0
            for idx in gm.validation_mask:
                print(f'--- Testing Week: {idx}')
                correct, total = Learning.evaluation(
                    model,
                    gm.graph_list,
                    [idx],
                    'RP',
                    return_counts=True
                )
                print(f'(Correct, Total): {(correct, total)}')
                t_correct += correct
                t_total += total

                train_list.append(idx)
                
                for epoch in range(epochs):
                    loss, train_acc = Learning.train(
                        model=model,
                        graph_list=gm.graph_list,
                        train_indcs=train_list,
                        mode='RP',
                        criterion=criterion,
                        optimizer=optimizer,
                    )

                    print(f'=================================== EPOCH {epoch + 1} ===================================')
                    print(f'Average Loss: {loss} - Train Accuracy: {train_acc: .3f}')

                    loss_list.append(loss)
                    train_acc_list.append(train_acc)


        except KeyboardInterrupt:
            pass
        current_val_acc = t_correct / t_total
        print(f'Total (Correct, Total): {(t_correct, t_total)} - Validation Accuracy: {current_val_acc: .3f}')
        chosen_epochs.loc[league, epochs] = current_val_acc
        torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_IncVal_{epochs}.pth')
        if current_val_acc > chosen_epochs.loc[league, 'accuracy']:
            chosen_epochs.loc[league, 'accuracy'] = current_val_acc
            chosen_epochs.loc[league, 'epochs'] = epochs
            torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_IncVal_best.pth')

Incremental Validation on league: England Premier League
--- Testing Week: 243
(Correct, Total): (6, 10)
--- Testing Week: 244
(Correct, Total): (5, 9)
--- Testing Week: 245
(Correct, Total): (7, 10)
--- Testing Week: 246
(Correct, Total): (3, 9)
--- Testing Week: 247
(Correct, Total): (1, 10)
--- Testing Week: 248
(Correct, Total): (6, 10)
--- Testing Week: 249
(Correct, Total): (5, 10)
--- Testing Week: 250
(Correct, Total): (6, 9)
--- Testing Week: 251
(Correct, Total): (2, 10)
--- Testing Week: 252
(Correct, Total): (6, 10)
--- Testing Week: 253
(Correct, Total): (4, 10)
--- Testing Week: 254
(Correct, Total): (8, 10)
--- Testing Week: 255
(Correct, Total): (8, 10)
--- Testing Week: 256
(Correct, Total): (6, 10)
--- Testing Week: 257
(Correct, Total): (7, 10)
--- Testing Week: 258
(Correct, Total): (7, 10)
--- Testing Week: 259
(Correct, Total): (5, 10)
--- Testing Week: 260
(Correct, Total): (5, 10)
--- Testing Week: 261
(Correct, Total): (3, 10)
--- Testing Week: 262
(Correct, To

In [47]:
chosen_epochs

,epochs,accuracy,0,1,2,3
England Premier League,0,0.527211,0.527211,0.493197,0.500000,0.489796
France Ligue 1,0,0.521127,0.521127,0.510563,0.503521,0.492958
Italy Serie A,0,0.501859,0.501859,0.494424,0.501859,0.501859
Spain LIGA BBVA,0,0.554348,0.554348,0.536232,0.536232,0.543478
Germany 1. Bundesliga,0,0.516667,0.516667,0.504167,0.500000,0.491667
Netherlands Eredivisie,0,0.540670,0.540670,0.526316,0.531100,0.526316
Scotland Premier League,0,0.585987,0.585987,0.579618,0.554140,0.554140
Portugal Liga ZON Sagres,0,0.542484,0.542484,0.529412,0.509804,0.535948
Belgium Jupiler League,0,0.539568,0.539568,0.517986,0.510791,0.525180


In [48]:
chosen_posit = chosen_epochs.apply(lambda z: z.iloc[3:].idxmax(), axis=1)
chosen_posit

England Premier League      2
France Ligue 1              1
Italy Serie A               2
Spain LIGA BBVA             3
Germany 1. Bundesliga       1
Netherlands Eredivisie      2
Scotland Premier League     1
Portugal Liga ZON Sagres    3
Belgium Jupiler League      3
dtype: int64

In [ ]:
# print('*********************************** Phase 2 ***********************************')
# for lig, (best_R, best_type) in eval_acc_df.apply(lambda z: z.idxmax(), axis=1).iteritems():
#     model = torch.load(f'{GLOBALS.SavePath.value}model_{lig if best_type in ["League", "MaxVal"] else "All"}_{best_R}{"_MaxVal" if best_type == "MaxVal" else ""}_Ph1.pth')
#     optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
#     print(f'--------- Appending validation to League: {lig} - Chosen Model: {(best_R, best_type)}')
#     if GLOBALS.already_saved.value:
#         gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
#     else:
#         gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
#     try:
#         max_acc = 0
#         train_indcs = gm.train_mask + gm.validation_mask
#         for epoch in range(20):
#             loss, train_acc = Learning.train(
#                 model=model,
#                 graph_list=gm.graph_list,
#                 train_indcs=train_indcs,
#                 mode='RP',
#                 criterion=criterion,
#                 optimizer=optimizer,
#             )

#             print(f'=================================== EPOCH {epoch + 1} ===================================')
#             print(f'Average Loss: {loss} - Train Accuracy: {train_acc: .3f}')

#             loss_list.append(loss)
#             train_acc_list.append(train_acc)

#             # if val_acc >= max_acc:
#             #     max_acc = val_acc
#             #     torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_MaxVal_Ph1.pth')

#             if (epoch+1) % GLOBALS.SaveEvery.value == 0:
#                 torch.save(model, f'{GLOBALS.SavePath.value}model.pth')
#                 with open(f'{GLOBALS.SavePath.value}lists.pl', 'wb') as pf:
#                     pickle.dump((loss_list, train_acc_list, eval_acc_list), pf)
#     except KeyboardInterrupt:
#         pass
#     torch.save(model, f'{GLOBALS.SavePath.value}model_{lig}_Ph2.pth')

In [58]:
print('*********************************** Phase 2 ***********************************')
for lig, (best_R, best_type) in eval_acc_df.apply(lambda z: z.idxmax(), axis=1).iteritems():
    model = torch.load(f'{GLOBALS.SavePath.value}model_{lig if best_type in ["League", "MaxVal"] else "All"}_{best_R}{"_MaxVal" if best_type == "MaxVal" else ""}_Ph1.pth')
    optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
    print(f'--------- Appending validation to League: {lig} - Chosen Model: {(best_R, best_type)}')
    
    for round in range(HYPERPARAETERS.Phase2Rounds.value):
        print(f'############################## Round {round + 1} ##############################')
        for league, league_df in dl.dataset.groupby('league', sort=False):
            print(f'Training On: {league}')
            if GLOBALS.already_saved.value:
                gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
            else:
                gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
            try:
                max_acc = 0
                train_indcs = gm.train_mask + gm.validation_mask
                for epoch in range(HYPERPARAETERS.Phase2Epochs.value):
                    loss, train_acc = Learning.train(
                        model=model,
                        graph_list=gm.graph_list,
                        train_indcs=train_indcs,
                        mode='RP',
                        criterion=criterion,
                        optimizer=optimizer,
                    )

                    print(f'=================================== EPOCH {epoch + 1} ===================================')
                    print(f'Average Loss: {loss} - Train Accuracy: {train_acc: .3f}')

                    loss_list.append(loss)
                    train_acc_list.append(train_acc)

                    # if val_acc >= max_acc:
                    #     max_acc = val_acc
                    #     torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_MaxVal_Ph1.pth')

                    if (epoch+1) % GLOBALS.SaveEvery.value == 0:
                        torch.save(model, f'{GLOBALS.SavePath.value}model.pth')
                        with open(f'{GLOBALS.SavePath.value}lists.pl', 'wb') as pf:
                            pickle.dump((loss_list, train_acc_list, eval_acc_list), pf)
            except KeyboardInterrupt:
                pass
            if f'R{round+1}' == best_R and best_type in ['League', 'MaxVal'] and league == lig:
                torch.save(model, f'{GLOBALS.SavePath.value}model_{lig}_Ph2.pth')
                break
        if f'R{round+1}' == best_R and best_type in ['League', 'MaxVal']: break
        if f'R{round+1}' == best_R and best_type == 'All':
            torch.save(model, f'{GLOBALS.SavePath.value}model_{lig}_Ph2.pth')
            break

*********************************** Phase 2 ***********************************
--------- Appending validation to League: England Premier League - Chosen Model: ('R1', 'MaxVal')
############################## Round 1 ##############################
Training On: England Premier League
=================================== EPOCH 1 ===================================
Average Loss: 1.0152318488547216 - Train Accuracy:  0.511
=================================== EPOCH 2 ===================================
Average Loss: 1.0099480169596688 - Train Accuracy:  0.508
=================================== EPOCH 3 ===================================
Average Loss: 1.006986597518781 - Train Accuracy:  0.507
=================================== EPOCH 4 ===================================
Average Loss: 1.0123122247147474 - Train Accuracy:  0.504
=================================== EPOCH 5 ===================================
Average Loss: 1.0098079052163569 - Train Accuracy:  0.509
===========================

In [49]:
chosen_incval_model = chosen_epochs.apply(lambda z: z.iloc[2:].idxmax(), axis=1)

In [59]:
t_acc_list = []
for league, league_df in dl.dataset.groupby('league', sort=False):
    print(f'Testing On: {league}')
    model = torch.load(f'{GLOBALS.SavePath.value}model_{league}_Ph2.pth')
    # model = torch.load(f'{GLOBALS.SavePath.value}model_{league}_IncVal_{chosen_incval_model[league]}.pth')
    optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
    if GLOBALS.already_saved.value:
        gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
    try:
        correct, total = Learning.evaluation(
            model,
            gm.graph_list,
            gm.test_mask,
            'RP',
            return_counts=True
        )
        t_acc_list.append((correct, total))
        print(f'Test Accuracy: {correct / total: .3f}')
    except KeyboardInterrupt:
        pass
print(f'Overal Accuracy: {np.sum(t_acc_list, axis=0)[0] / np.sum(t_acc_list, axis=0)[1]: .3f}')

Testing On: England Premier League
Test Accuracy:  0.431
Testing On: France Ligue 1
Test Accuracy:  0.491
Testing On: Italy Serie A
Test Accuracy:  0.529
Testing On: Spain LIGA BBVA
Test Accuracy:  0.552
Testing On: Germany 1. Bundesliga
Test Accuracy:  0.476
Testing On: Netherlands Eredivisie
Test Accuracy:  0.498
Testing On: Scotland Premier League
Test Accuracy:  0.456
Testing On: Portugal Liga ZON Sagres
Test Accuracy:  0.547
Testing On: Belgium Jupiler League
Test Accuracy:  0.475
Overal Accuracy:  0.495


In [65]:
print('*********************************** Phase 3 ***********************************')
test_correct_list = []
for league, league_df in dl.dataset.groupby('league', sort=False):
    print(f'Testing On: {league}')
    model = torch.load(f'{GLOBALS.SavePath.value}model_{league}_Ph2.pth')
    # model = torch.load(f'{GLOBALS.SavePath.value}model_{league}_IncVal_{chosen_incval_model[league]}.pth')
    optimizer = torch.optim.Adam(model.parameters(), lr=HYPERPARAETERS.LearningRate.value)
    if GLOBALS.already_saved.value:
        gm = GraphManager.load(f'{GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{GLOBALS.SavePath.value}{league}.gm')
    try:
        train_list = gm.train_mask + gm.validation_mask
        # train_list = []
        t_correct = 0
        t_total = 0
        for idx in gm.test_mask:
            print(f'--- Testing Week: {idx}')
            correct, total = Learning.evaluation(
                model,
                gm.graph_list,
                [idx],
                'RP',
                return_counts=True
            )
            print(f'(Correct, Total): {(correct, total)}')
            t_correct += correct
            t_total += total

            train_list.append(idx)
            
            for epoch in range(chosen_posit[league]):
                loss, train_acc = Learning.train(
                    model=model,
                    graph_list=gm.graph_list,
                    train_indcs=train_list,
                    mode='RP',
                    criterion=criterion,
                    optimizer=optimizer,
                )

                print(f'=================================== EPOCH {epoch + 1} ===================================')
                print(f'Average Loss: {loss} - Train Accuracy: {train_acc: .3f}')

                loss_list.append(loss)
                train_acc_list.append(train_acc)

    except KeyboardInterrupt:
        pass
    print(f'Total (Correct, Total): {(t_correct, t_total)} - Test Accuracy: {t_correct / t_total: .3f}')
    test_correct_list.append((t_correct, t_total))
    torch.save(model, f'{GLOBALS.SavePath.value}model_{league}_Ph3.pth')
    
with open(f'{GLOBALS.SavePath.value}test_list.pl', 'wb') as pf:
    pickle.dump(test_correct_list, pf)

*********************************** Phase 3 ***********************************
Testing On: England Premier League
--- Testing Week: 273
(Correct, Total): (3, 10)
=================================== EPOCH 1 ===================================
Average Loss: 0.9955848231367821 - Train Accuracy:  0.520
=================================== EPOCH 2 ===================================
Average Loss: 0.9969001014302247 - Train Accuracy:  0.513
--- Testing Week: 274
(Correct, Total): (5, 10)
=================================== EPOCH 1 ===================================
Average Loss: 0.9968058297850869 - Train Accuracy:  0.520
=================================== EPOCH 2 ===================================
Average Loss: 0.9964979143576188 - Train Accuracy:  0.510
--- Testing Week: 275
(Correct, Total): (4, 10)
=================================== EPOCH 1 ===================================
Average Loss: 0.9919846159392509 - Train Accuracy:  0.513
=================================== EPOCH 2 =======

In [66]:
test_acc = np.array(test_correct_list)
pd.DataFrame(test_acc[:, 0] / test_acc[:, 1], index=dl.dataset['league'].unique())

,0
England Premier League,0.454248
France Ligue 1,0.483986
Italy Serie A,0.540146
Spain LIGA BBVA,0.548611
Germany 1. Bundesliga,0.488095
Netherlands Eredivisie,0.507177
Scotland Premier League,0.481013
Portugal Liga ZON Sagres,0.558824
Belgium Jupiler League,0.460432


In [67]:
np.sum(test_acc, axis=0)[0] / np.sum(test_acc, axis=0)[1]

0.503129513721714

In [ ]:
#Run Everything Above

## Test Cells - DO NOT RUN

In [ ]:
#Results Prediction
print('*********************************** Phase 1 ***********************************')
Learning.Phase1(dl, model, criterion, optimizer, Learning.train_step, Learning.evaluation, already_saved, 2, 100, loss_list, train_acc_list, eval_acc_list)
print('*********************************** Phase 2 ***********************************')
Learning.Phase2(dl, model, criterion, optimizer, Learning.train_step, Learning.evaluation, already_saved, 1, 20, loss_list, train_acc_list, eval_acc_list)
print('*********************************** Phase 3 ***********************************')
Learning.Phase3(dl, model, criterion, optimizer, Learning.train_step, Learning.evaluation, already_saved, 1, loss_list, train_acc_list, eval_acc_list)

In [7]:
#Goal Diff
print('*********************************** Phase 1 ***********************************')
Learning.Phase1(dl, model, criterion, optimizer, Learning.goal_diff_train_step, Learning.goal_diff_evaluation, already_saved, 2, 100, loss_list, train_acc_list, eval_acc_list)
print('*********************************** Phase 2 ***********************************')
# Learning.Phase2(dl, model, criterion, optimizer, Learning.goal_diff_train_step, Learning.goal_diff_evaluation, already_saved, 1, 20, loss_list, train_acc_list, eval_acc_list)
print('*********************************** Phase 3 ***********************************')
# Learning.Phase3(dl, model, criterion, optimizer, Learning.goal_diff_train_step, Learning.goal_diff_evaluation, already_saved, 1, loss_list, train_acc_list, eval_acc_list)

*********************************** Phase 1 ***********************************
############################## Round 1 ##############################
Training On: Belgium Jupiler League
=================================== EPOCH 1 ===================================
Average Loss: 3.3410798667106203 - Train Accuracy:  0.291
Validation Accuracy:  0.254
=================================== EPOCH 2 ===================================
Average Loss: 3.2804844480676527 - Train Accuracy:  0.303
Validation Accuracy:  0.254
=================================== EPOCH 3 ===================================
Average Loss: 3.160364084280072 - Train Accuracy:  0.297
Validation Accuracy:  0.254
=================================== EPOCH 4 ===================================
Average Loss: 3.0729300072655463 - Train Accuracy:  0.329
Validation Accuracy:  0.254
=================================== EPOCH 5 ===================================
Average Loss: 3.1245527452048965 - Train Accuracy:  0.285
Validation Ac

In [6]:
model = torch.load(f'Runs/26Jan_2/model.pth')

In [6]:
for league, league_df in dl.dataset.groupby('league'):
    print(f'Evaluating On: {league}')
    if already_saved:
        gm = GraphManager.load(f'{Utils.GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{Utils.GLOBALS.SavePath.value}{league}.gm')
    
    t_correct = 0
    t_total = 0

    for idx in gm.validation_mask:
        g = gm.graph_list[idx]
        correct, total = Learning.goal_diff_evaluation(model, g)
        t_correct += correct
        t_total += total
    print(f'Test Accuracy: {t_correct / t_total: .3f}')

Evaluating On: Belgium Jupiler League
Test Accuracy:  0.415
Evaluating On: England Premier League
Test Accuracy:  0.407
Evaluating On: France Ligue 1
Test Accuracy:  0.409
Evaluating On: Germany 1. Bundesliga
Test Accuracy:  0.346
Evaluating On: Italy Serie A
Test Accuracy:  0.419
Evaluating On: Netherlands Eredivisie
Test Accuracy:  0.415
Evaluating On: Poland Ekstraklasa
Test Accuracy:  0.324
Evaluating On: Portugal Liga ZON Sagres
Test Accuracy:  0.391
Evaluating On: Scotland Premier League
Test Accuracy:  0.308
Evaluating On: Spain LIGA BBVA
Test Accuracy:  0.408
Evaluating On: Switzerland Super League
Test Accuracy:  0.386


In [7]:
for league, league_df in dl.dataset.groupby('league'):
    print(f'Testing On: {league}')
    if already_saved:
        gm = GraphManager.load(f'{Utils.GLOBALS.LoadPath.value}{league}.gm')
    else:
        gm = GraphManager.load(f'{Utils.GLOBALS.SavePath.value}{league}.gm')
    
    t_correct = 0
    t_total = 0

    for idx in gm.test_mask:
        g = gm.graph_list[idx]
        correct, total = evaluation(model, g)
        t_correct += correct
        t_total += total
    print(f'Test Accuracy: {t_correct / t_total: .3f}')

Testing On: Belgium Jupiler League
Test Accuracy:  0.452
Testing On: England Premier League
Test Accuracy:  0.433
Testing On: France Ligue 1
Test Accuracy:  0.486
Testing On: Germany 1. Bundesliga
Test Accuracy:  0.484
Testing On: Italy Serie A
Test Accuracy:  0.500
Testing On: Netherlands Eredivisie
Test Accuracy:  0.495
Testing On: Poland Ekstraklasa
Test Accuracy:  0.448
Testing On: Portugal Liga ZON Sagres
Test Accuracy:  0.539
Testing On: Scotland Premier League
Test Accuracy:  0.513
Testing On: Spain LIGA BBVA
Test Accuracy:  0.524
Testing On: Switzerland Super League
Test Accuracy:  0.551


In [13]:
loss

428

In [8]:
model(gm.graph_list[3])

tensor([[-2.7374e+00, -8.8250e+00, -6.7085e-02],
        [-1.1629e+01, -2.9417e+01, -8.9407e-06]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)

In [8]:
dl.dataset.loc[dl.dataset['league'] == 'Belgium Jupiler League', :]

,league,season,week,home_team,away_team,result,home_lineup,away_lineup
0,Belgium Jupiler League,2008/2009,24,KV Mechelen,KRC Genk,win,"[Wouter Biebauw, Kenny van Hoevelen, Nana Asar...","[Davino Verhulst, Joao Carlos, Dimitri Daesela..."
1,Belgium Jupiler League,2008/2009,25,KSV Cercle Brugge,Club Brugge KV,loss,"[Bram Verbist, Denis Viane, Anthony Portier, F...","[Stijn Stijnen, Michael Klukowski, Antolin Alc..."
2,Belgium Jupiler League,2008/2009,25,RSC Anderlecht,SV Zulte-Waregem,win,"[Davy Schollen, Olivier Deschacht, Arnold Krui...","[Sammy Bossuyt, Karel D'Haene, Stijn Minne, Ba..."
3,Belgium Jupiler League,2008/2009,26,KV Mechelen,RSC Anderlecht,win,"[Wouter Biebauw, Kenny van Hoevelen, Nana Asar...","[Davy Schollen, Olivier Deschacht, Roland Juha..."
4,Belgium Jupiler League,2008/2009,26,SV Zulte-Waregem,KSV Roeselare,tie,"[Sammy Bossuyt, Karel D'Haene, Steve Colpaert,...","[Jurgen Sierens, Damir Mirvic, Mladen Lazarevi..."
...,...,...,...,...,...,...,...,...
1209,Belgium Jupiler League,2015/2016,30,SV Zulte-Waregem,Royal Excel Mouscron,win,"[Kenny Steppe, Henrik Dalsgaard, Christophe Le...","[Vagner, Noe Dussenne, Mickael Tirpan, Jean-Ch..."
1210,Belgium Jupiler League,2015/2016,30,Sporting Charleroi,KAA Gent,tie,"[Nicolas Penneteau, Damien Marcq, Gjoko Zajkov...","[Matz Sels, Lasse Nielsen, Rami Gershon, Nana ..."
1211,Belgium Jupiler League,2015/2016,30,Oud-Heverlee Leuven,Club Brugge KV,loss,"[Rudy Riou, Jordan Remacle, Kanu, Romain Reyna...","[Ludovic Butelle, Thomas Meunier, Bjorn Engels..."
1212,Belgium Jupiler League,2015/2016,30,KVC Westerlo,Waasland-Beveren,win,"[Kristof van Hout, Mitch Apau, Kenneth Schuerm...","[Laurent Henkinet, Hugo Sousa, Gary Coulibaly,..."


In [9]:
ht = gm._gen_heterodata(dl.dataset.loc[dl.dataset['league'] == 'Belgium Jupiler League', :], supervision_indcs=dl.dataset.loc[dl.dataset['league'] == 'Belgium Jupiler League', :].index[-20:])

In [10]:
ht

HeteroData(
  team={ x=[2428] },
  player={ x=[26708] },
  (team, used, player)={ edge_index=[2, 26708] },
  (player, playedin, team)={ edge_index=[2, 26708] },
  (team, win, team)={ edge_index=[2, 899] },
  (team, loss, team)={ edge_index=[2, 899] },
  (team, tie, team)={ edge_index=[2, 590] },
  (team, before, team)={ edge_index=[2, 2406] },
  (team, after, team)={ edge_index=[2, 2406] },
  (player, before, player)={ edge_index=[2, 25641] },
  (player, after, player)={ edge_index=[2, 25641] }
)

In [8]:
ht.to_homogeneous()

Data(node_type=[96], x=[96], edge_index=[2, 184], edge_type=[184])

In [9]:
hm = ht.to_homogeneous()
g = torch_geometric.utils.to_networkx(hm)
pos = nx.spring_layout(g)  # positions for all nodes


node_labels = {}
for i, n in enumerate(hm.node_type.cpu().numpy()):
    node_labels[i] = 'team' if n==0 else 'player'

edge_list = hm.edge_index.T.cpu().tolist()
edge_labels = {}
for i, e in enumerate(hm.edge_type.cpu().numpy()):
    if e==0 or e==1: edge_labels[tuple(edge_list[i])] = 'used - played_in'
    elif e==2 or e==3: edge_labels[tuple(edge_list[i])] = 'won - loss'

fig, ax = plt.subplots()
nx.draw_networkx_nodes(
    g,
    ax=ax,
    pos=pos)
nx.draw_networkx_labels(g,pos=pos, ax=ax, labels=node_labels)
nx.draw_networkx_edges(g, pos, ax=ax, connectionstyle='arc3,rad=0.1')
nx.draw_networkx_edge_labels(g, pos=pos, ax=ax, edge_labels=edge_labels)
fig.set_size_inches((100, 100))
fig.savefig('a.png')

NameError: name 'ht' is not defined

In [7]:
for i, e in enumerate(hm.edge_index.cpu().numpy().T):
    print(i)
    print(e)
    break

0
[0 2]


In [8]:
np.moveaxis(np.stack((
            dl.DatasetDataframetoNumpy(dl.dataset)[2],
            dl.DatasetDataframetoNumpy(dl.dataset)[3]
        )), 0, 1).shape


(21309, 2, 11)

In [9]:
dl.DatasetDataframetoNumpy(dl.dataset)[2].shape

(21309, 11)

In [10]:
team_node_features = dl.labeler.transform(np.moveaxis(np.stack((
            dl.DatasetDataframetoNumpy(dl.dataset)[2],
            dl.DatasetDataframetoNumpy(dl.dataset)[3]
        )), 0, 1).flatten())

In [11]:
for i, r in dl.dataset.iterrows():
    print(r['week'])
    break


24


In [12]:
np.stack((
    dl.DatasetDataframetoNodeText(dl.dataset)[0],
    dl.DatasetDataframetoNodeText(dl.dataset)[1]
    )).T.flatten()

array(['KV Mechelen*0', 'KRC Genk*0', 'KSV Cercle Brugge*1', ...,
       'BSC Young Boys*21307', 'FC Zürich*21308', 'FC Vaduz*21308'],
      dtype='<U34')

In [13]:
a = dl.DatasetDataframetoNodeText(dl.dataset)
np.arange(a[0].shape[0] * 2)
t = pd.Series(
    np.arange(a[0].shape[0] * 2),
    index=np.stack((
        dl.DatasetDataframetoNodeText(dl.dataset)[0],
        dl.DatasetDataframetoNodeText(dl.dataset)[1]
    )).T.flatten()
)

In [14]:
p = pd.Series(
    np.arange(a[2].shape[0] * a[2].shape[1] * 2),
    index=np.moveaxis(np.stack((
        dl.DatasetDataframetoNodeText(dl.dataset)[2],
        dl.DatasetDataframetoNodeText(dl.dataset)[3]
    )), 0, 1).flatten())

In [15]:
p.to_numpy()

array([     0,      1,      2, ..., 468795, 468796, 468797])

In [16]:
aaa=np.repeat(t.to_numpy(), dl.minimum_players_per_team)

In [17]:
ttt = torch.stack((
    torch.tensor(aaa),
    torch.tensor(p)
)).T

In [18]:
aaa.shape

(468798,)

In [19]:
np.arange(a[2].shape[0] * a[2].shape[1] * 2).reshape(a[2].shape[0], -1, a[2].shape[1])

array([[[     0,      1,      2, ...,      8,      9,     10],
        [    11,     12,     13, ...,     19,     20,     21]],

       [[    22,     23,     24, ...,     30,     31,     32],
        [    33,     34,     35, ...,     41,     42,     43]],

       [[    44,     45,     46, ...,     52,     53,     54],
        [    55,     56,     57, ...,     63,     64,     65]],

       ...,

       [[468732, 468733, 468734, ..., 468740, 468741, 468742],
        [468743, 468744, 468745, ..., 468751, 468752, 468753]],

       [[468754, 468755, 468756, ..., 468762, 468763, 468764],
        [468765, 468766, 468767, ..., 468773, 468774, 468775]],

       [[468776, 468777, 468778, ..., 468784, 468785, 468786],
        [468787, 468788, 468789, ..., 468795, 468796, 468797]]])

In [20]:
dl.dataset.loc[dl.dataset['result']=='win', :].index.values

array([    0,     2,     3, ..., 21297, 21305, 21308])

In [22]:
dl.DatasetDataframetoNodeText(
                dl.dataset.loc[dl.dataset['result'] == 'win', :]
            )

(array(['KV Mechelen*0', 'RSC Anderlecht*2', 'KV Mechelen*3', ...,
        'FC St. Gallen*21297', 'Lugano*21305', 'FC Zürich*21308'],
       dtype='<U34'),
 array(['KRC Genk*0', 'SV Zulte-Waregem*2', 'RSC Anderlecht*3', ...,
        'FC Zürich*21297', 'FC St. Gallen*21305', 'FC Vaduz*21308'],
       dtype='<U34'),
 array([['Wouter Biebauw@0', 'Kenny van Hoevelen@0', 'Nana Asare@0', ...,
         'Romeo van Dessel@0', 'Wouter Vrancken@0', 'Giuseppe Rossini@0'],
        ['Davy Schollen@2', 'Olivier Deschacht@2', 'Arnold Kruiswijk@2',
         ..., 'Mbark Boussoufa@2', 'Oleksandr Iakovenko@2',
         'Tom De Sutter@2'],
        ['Wouter Biebauw@3', 'Kenny van Hoevelen@3', 'Nana Asare@3', ...,
         'Romeo van Dessel@3', 'Bjoern Vleminckx@3', 'Giuseppe Rossini@3'],
        ...,
        ['Daniel Lopar@21297', 'Alain Wiss@21297', 'Silvan Hefti@21297',
         ..., 'Danijel Aleksic@21297', 'Marco Aratore@21297',
         'Edgar Salli@21297'],
        ['Mirko Salvi@21305', 'Frederic Vese